In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install einops yacs timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
!git clone https://github.com/swz30/Restormer.git
%cd Restormer


Cloning into 'Restormer'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 309 (delta 67), reused 56 (delta 56), pack-reused 202 (from 1)
Receiving objects: 100% (309/309), 1.56 MiB | 4.39 MiB/s, done.
Resolving deltas: 100% (123/123), done.
/content/Restormer


In [ ]:
import zipfile

zip_path = "/content/drive/My Drive/Image_Sharpening/blur_patches.zip"
extract_dir = "/content/blur_patches"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ blur patches unzipped!")


✅ blur patches unzipped!


In [ ]:
!pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 6.3 MB/s eta 0:00:00


In [ ]:
import os
import sys
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

# === Config ===
drive_root = '/content/drive/MyDrive/Image_Sharpening'
input_folder = "/content/blur_patches/blur_patches"
output_folder = os.path.join(drive_root, 'teacher_patches_dual')
os.makedirs(output_folder, exist_ok=True)

# Paths to pretrained models
defocus_path = os.path.join(drive_root, 'single_image_defocus_deblurring.pth')
denoise_path = os.path.join(drive_root, 'gaussian_color_denoising_blind.pth')

# Add Restormer repo path
restormer_root = '/content/Restormer'
sys.path.insert(0, restormer_root)

from basicsr.models.archs.restormer_arch import Restormer

# === Setup ===
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

# === Load Defocus Teacher ===
model_def = Restormer().to(device)
ckpt_def = torch.load(defocus_path, map_location=device)
model_def.load_state_dict(ckpt_def['params'], strict=False)
model_def.eval()

# === Load Denoise Teacher ===
model_den = Restormer().to(device)
ckpt_den = torch.load(denoise_path, map_location=device)
model_den.load_state_dict(ckpt_den['params'], strict=False)
model_den.eval()

# === Weighted Average Settings ===
weight_def, weight_den = 0.7, 0.3

# === Process Blurred Patches ===
png_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.png')]

with torch.no_grad():
    for fname in tqdm(png_files, desc="Generating dual-teacher soft targets"):
        img_path = os.path.join(input_folder, fname)
        img = Image.open(img_path).convert("RGB")
        inp = to_tensor(img).unsqueeze(0).to(device)

        # Pad to multiple of 8
        h, w = inp.shape[2:]
        pad_h = (8 - h % 8) % 8
        pad_w = (8 - w % 8) % 8
        inp_padded = F.pad(inp, (0, pad_w, 0, pad_h), mode='reflect')

        # Inference from both teachers
        out_def = model_def(inp_padded)[:, :, :h, :w]
        out_den = model_den(inp_padded)[:, :, :h, :w]

        # Weighted fusion
        soft_target = torch.clamp(weight_def * out_def + weight_den * out_den, 0, 1)

        # Save PNG
        out_img = to_pil(soft_target.squeeze(0).cpu())
        out_img.save(os.path.join(output_folder, fname))

print("✅ All soft targets generated using both teachers (saved as PNG).")


Generating dual-teacher soft targets: 100%|██████████| 5413/5413 [46:11<00:00,  1.95it/s]

✅ All soft targets generated using both teachers (saved as PNG).
